In [26]:
import collections
import csv
import os
import random
import tarfile
import torch
from torch import nn
import torchtext.vocab as Vocab
import torch.utils.data as Data
import time
import sys 
import d2lzh_pytorch as d2l
sys.path.append("..") 
DATA_ROOT = "/dataset"

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [27]:
lines = open("dataset/labeledTrainData.tsv",encoding ='utf-8')
data=lines.read().strip().split('\n')
data=data[1:]
random.shuffle(data)

In [28]:
n=len(data)
feature=[]
label=[]
for i in range(n):
    line=data[i]
    ts=line.strip().split('\t')
    label.append(int(ts[1]))
    feature.append(ts[2])


In [29]:
def get_tokenized_imdb(data):
    def tokenizer(text):
        return [tok.lower() for tok in text.split(' ')]
    return [tokenizer(review) for review in data]

def get_vocab_imdb(data):
    tokenized_data = get_tokenized_imdb(data)
    counter = collections.Counter([tk for st in tokenized_data for tk in st])
    return Vocab.Vocab(counter, min_freq=5)

vocab = get_vocab_imdb(feature)
'# words in vocab:', len(vocab)

('# words in vocab:', 46790)

In [30]:
def preprocess_imdb(feature,vocab):
    max_l = 500 

    def pad(x):
        return x[:max_l] if len(x) > max_l else x + [0] * (max_l - len(x))

    tokenized_data = get_tokenized_imdb(feature)
    features = [pad([vocab.stoi[word] for word in words]) for words in tokenized_data]
    return features

In [31]:
batch_size = 64
features=preprocess_imdb(feature,vocab)
trlabel=torch.tensor(label[:int(n*4/5)])
trfeature=torch.tensor(features[:int(n*4/5)])
telabel=torch.tensor(label[int(n*4/5):])
tefeature=torch.tensor(features[int(n*4/5):])

train_set = Data.TensorDataset(trfeature,trlabel)
test_set = Data.TensorDataset(tefeature,telabel)
train_iter = Data.DataLoader(train_set, batch_size, shuffle=True)
test_iter = Data.DataLoader(test_set, batch_size)

In [32]:
for X, y in train_iter:
    print('X', X.shape, 'y', y.shape)
    break
'#batches:', len(train_iter)


X torch.Size([64, 500]) y torch.Size([64])


('#batches:', 313)

In [33]:
class BiRNN(nn.Module):
    def __init__(self, vocab, embed_size, num_hiddens, num_layers):
        super(BiRNN, self).__init__()
        self.embedding = nn.Embedding(len(vocab), embed_size)
        self.encoder = nn.LSTM(input_size=embed_size, 
                                hidden_size=num_hiddens, 
                                num_layers=num_layers,
                                bidirectional=True)
        self.decoder = nn.Linear(4*num_hiddens, 2)

    def forward(self, inputs):
        
        embeddings = self.embedding(inputs.permute(1, 0))
        outputs, _ = self.encoder(embeddings) 

        encoding = torch.cat((outputs[0], outputs[-1]), -1)
        outs = self.decoder(encoding)
        return outs

In [34]:
embed_size, num_hiddens, num_layers = 100, 100, 2
net = BiRNN(vocab, embed_size, num_hiddens, num_layers)

In [35]:
glove_vocab = Vocab.GloVe(name='6B', dim=100, cache=os.path.join(DATA_ROOT, "glove"))
def load_pretrained_embedding(words, pretrained_vocab):
    """从预训练好的vocab中提取出words对应的词向量"""
    embed = torch.zeros(len(words), pretrained_vocab.vectors[0].shape[0]) # 初始化为0
    oov_count = 0 # out of vocabulary
    for i, word in enumerate(words):
        try:
            idx = pretrained_vocab.stoi[word]
            embed[i, :] = pretrained_vocab.vectors[idx]
        except KeyError:
            oov_count += 1
    if oov_count > 0:
        print("There are %d oov words." % oov_count)
    return embed

net.embedding.weight.data.copy_(
    load_pretrained_embedding(vocab.itos, glove_vocab))
net.embedding.weight.requires_grad = False # 直接加载预训练好的, 所以不需要更新它

There are 21883 oov words.


In [11]:
lr, num_epochs = 0.01, 5
# 要过滤掉不计算梯度的embedding参数
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), lr=lr)
loss = nn.CrossEntropyLoss()
d2l.train(train_iter, test_iter, net, loss, optimizer, device, num_epochs)

training on  cpu
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273

In [14]:
torch.save(net, 'net.pkl')

In [36]:
net=torch.load('net.pkl')
def predict_sentiment(net, vocab, sentence):
    """sentence是词语的列表"""
    device = list(net.parameters())[0].device
    sentence = torch.tensor([vocab.stoi[word] for word in sentence], device=device)
    label = torch.argmax(net(sentence.view((1, -1))), dim=1)
    return label.item()
predict_sentiment(net, vocab, ['this', 'movie', 'is', 'so', 'great'])

1

In [37]:
testlines = open("dataset/testData.tsv",encoding ='utf-8')
testdata=testlines.read().strip().split('\n')
testdata=testdata[1:]
n=len(testdata)
ID=[]
tf=[]
for i in range(n):
    line=testdata[i]
    ts=line.strip().split('\t')
    ID.append(ts[0])
    tf.append(ts[1].strip().split(' '))


In [38]:
f = open('dataset/Submission.csv','w')
writer = csv.writer(f)
for i in range(n):
    writer.writerow((ID[i], predict_sentiment(net, vocab, tf[i])))
f.close()